In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import nltk
import gensim
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer

import joblib

from sklearn.metrics import accuracy_score,confusion_matrix, classification_report

In [2]:
dataset = pd.read_csv('../Datasets/After_preprocessing.csv')

In [3]:
dataset.head()

,Unnamed: 0,question1,question2,is_duplicate,Q1_cleaned,Q2_cleaned
0,320645,I want to increase my height.I have started dr...,Which is the best subject to take in civil ove...,0,want increase height started drinking l milk j...,best subject take civil overseas
1,391991,How do I delete my Snapchat conversations in b...,If you delete your snapchat does it delete the...,0,delete snapchat conversations blue person saved,delete snapchat delete saved messages someone ...
2,175411,How one can deal with a breakup?,What are the steps one goes through getting ov...,1,one deal breakup,steps one goes getting breakup
3,42832,Why is Pope John Paul II becoming a saint?,Does Pope John Paul II deserve sainthood?,0,pope john paul ii becoming saint,pope john paul ii deserve sainthood
4,212324,What are some best anime?,"What are some of the best Japanese anime, and ...",0,best anime,best japanese anime


In [4]:
dataset.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
dataset.head()

,question1,question2,is_duplicate,Q1_cleaned,Q2_cleaned
0,I want to increase my height.I have started dr...,Which is the best subject to take in civil ove...,0,want increase height started drinking l milk j...,best subject take civil overseas
1,How do I delete my Snapchat conversations in b...,If you delete your snapchat does it delete the...,0,delete snapchat conversations blue person saved,delete snapchat delete saved messages someone ...
2,How one can deal with a breakup?,What are the steps one goes through getting ov...,1,one deal breakup,steps one goes getting breakup
3,Why is Pope John Paul II becoming a saint?,Does Pope John Paul II deserve sainthood?,0,pope john paul ii becoming saint,pope john paul ii deserve sainthood
4,What are some best anime?,"What are some of the best Japanese anime, and ...",0,best anime,best japanese anime


In [6]:
dataset.shape

(30000, 5)

## Vectorizing Text Data

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# # merge texts
questions = list(dataset['Q1_cleaned']) + list(dataset['Q2_cleaned'])

# This code is to vectorize using Word2Vec - START

# model = Word2Vec(window=10, min_count=2, workers=8, vector_size=300)
# model.build_vocab(questions, progress_per=1000)
# model.train(questions, total_examples=model.corpus_count, epochs=model.epochs)

# This code is to vectorize using Word2Vec - END

cv = TfidfVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
temp_df1 = pd.DataFrame(q1_arr, index= dataset.index, columns=list(range(0,3000)))
temp_df2 = pd.DataFrame(q2_arr, index= dataset.index, columns=list(range(3001,6001)))
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

In [ ]:
temp_df1.head()

In [ ]:
temp_df2.head()

In [ ]:
temp_df.head()

In [ ]:
temp_df['is_duplicate'] = dataset['is_duplicate']

In [ ]:
temp_df.head()

## Adding some more artificial features

In [ ]:
dataset.head()

,question1,question2,is_duplicate,Q1_cleaned,Q2_cleaned
320645,I want to increase my height.I have started dr...,Which is the best subject to take in civil ove...,0,want increase height started drinking l milk j...,best subject take civil overseas
391991,How do I delete my Snapchat conversations in b...,If you delete your snapchat does it delete the...,0,delete snapchat conversations blue person saved,delete snapchat delete saved messages someone ...
175411,How one can deal with a breakup?,What are the steps one goes through getting ov...,1,one deal breakup,steps one goes getting breakup
42832,Why is Pope John Paul II becoming a saint?,Does Pope John Paul II deserve sainthood?,0,pope john paul ii becoming saint,pope john paul ii deserve sainthood
212324,What are some best anime?,"What are some of the best Japanese anime, and ...",0,best anime,best japanese anime


In [ ]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,5992,5993,5994,5995,5996,5997,5998,5999,6000,is_duplicate
320645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
391991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
175411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
42832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
212324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
def getUnique(arr):
  return len(set(arr.split(' ')))

In [ ]:
def getCommon(arr1, arr2):

  set1 = set(arr1.split(' '))
  set2 = set(arr2.split(' '))
  return len(set1 & set2)

In [ ]:
def wordsTotal(arr1, arr2):

  set1 = set(arr1.split(' '))
  set2 = set(arr2.split(' '))
  return len(set1) + len(set2)

In [ ]:
# Lengths of each sentences
q1_len = [len(sentence) for sentence in dataset['Q1_cleaned']]
q2_len = [len(sentence) for sentence in dataset['Q2_cleaned']]

# Number of words in each sentences
q1_words = [len(arr.split(' ')) for arr in dataset['Q1_cleaned']]
q2_words = [len(arr.split(' ')) for arr in dataset['Q2_cleaned']]

common_words = [getCommon(arr1, arr2) for arr1, arr2 in zip(dataset['Q1_cleaned'], dataset['Q2_cleaned'])]

total_words = [wordsTotal(arr1, arr2) for arr1, arr2 in zip(dataset['Q1_cleaned'], dataset['Q2_cleaned'])]

In [ ]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,5992,5993,5994,5995,5996,5997,5998,5999,6000,is_duplicate
320645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
391991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
175411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
42832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
212324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
temp_df['q1_len'] = q1_len
temp_df['q2_len'] = q2_len

temp_df['q1_words'] = q1_words
temp_df['q2_words'] = q2_words

temp_df['common_words'] = common_words
temp_df['total_words'] = total_words

temp_df['words_share'] = round(temp_df['common_words']/temp_df['total_words'],2)

In [ ]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,5999,6000,is_duplicate,q1_len,q2_len,q1_words,q2_words,common_words,total_words,words_share
320645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,88,32,13,5,0,16,0.00
391991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,47,56,6,8,3,12,0.25
175411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,16,30,3,5,2,8,0.25
42832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,32,35,6,6,4,12,0.33
212324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,10,19,2,3,2,5,0.40


## Adding Fuzzy Features

In [ ]:
import fuzzywuzzy
from fuzzywuzzy import fuzz

C:\Users\Priyesh Dave\AppData\Local\Programs\Python\Python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
tokenSet = list()
tokenSort = list()
partialTokenSet = list()
partialTokenSort = list()

In [ ]:
def getTokenSet(df):
  tokenSet.append(fuzz.token_set_ratio(df['Q1_cleaned'], df['Q2_cleaned']))
  tokenSort.append(fuzz.token_sort_ratio(df['Q1_cleaned'], df['Q2_cleaned']))
  partialTokenSet.append(fuzz.partial_token_set_ratio(df['Q1_cleaned'], df['Q2_cleaned']))
  partialTokenSort.append(fuzz.partial_token_sort_ratio(df['Q1_cleaned'], df['Q2_cleaned']))

In [ ]:
dataset.apply(getTokenSet, axis=1)

320645    None
391991    None
175411    None
42832     None
212324    None
          ... 
97053     None
80954     None
103277    None
341469    None
400381    None
Length: 30000, dtype: object

In [ ]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,5999,6000,is_duplicate,q1_len,q2_len,q1_words,q2_words,common_words,total_words,words_share
320645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,88,32,13,5,0,16,0.00
391991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,47,56,6,8,3,12,0.25
175411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,16,30,3,5,2,8,0.25
42832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,32,35,6,6,4,12,0.33
212324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,10,19,2,3,2,5,0.40


In [ ]:
temp_df['token_set'] = tokenSet
temp_df['token_sort'] = tokenSort
temp_df['partial_token_set'] = partialTokenSet
temp_df['partial_token_sort'] = partialTokenSort

In [ ]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,q2_len,q1_words,q2_words,common_words,total_words,words_share,token_set,token_sort,partial_token_set,partial_token_sort
320645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,32,13,5,0,16,0.00,27,23,38,38
391991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,56,6,8,3,12,0.25,66,54,100,60
175411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,30,3,5,2,8,0.25,81,57,100,62
42832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,35,6,6,4,12,0.33,75,75,100,78
212324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,19,2,3,2,5,0.40,100,69,100,100


## Model Training

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X_train_advance, X_test_advance, y_train_advance, y_test_advance = train_test_split(temp_df.drop(['is_duplicate'], axis=1), temp_df['is_duplicate'], test_size=.2, random_state=2)

In [ ]:
xgb_advance = XGBClassifier()
xgb_advance.fit(X_train_advance, y_train_advance)

pred_train = xgb_advance.predict(X_train_advance)
pred_test = xgb_advance.predict(X_test_advance)

C:\Users\Priyesh Dave\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:54:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7608333333333334

In [ ]:
print('Training Accuracy:', accuracy_score(y_train_advance, pred_train))
print('Validation Accuracy:', accuracy_score(y_test_advance, pred_test))

Training Accuracy: 0.8339583333333334
Validation Accuracy: 0.7608333333333334


In [ ]:
print(classification_report(y_train_advance, pred_train))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87     15164
           1       0.81      0.72      0.76      8836

    accuracy                           0.83     24000
   macro avg       0.83      0.81      0.82     24000
weighted avg       0.83      0.83      0.83     24000



In [ ]:
print(classification_report(y_test_advance, pred_test))

              precision    recall  f1-score   support

           0       0.79      0.86      0.82      3798
           1       0.71      0.60      0.65      2202

    accuracy                           0.76      6000
   macro avg       0.75      0.73      0.73      6000
weighted avg       0.76      0.76      0.76      6000



In [ ]:
joblib.dump(xgb_advance, '../Model/xgb_advance.pkl')

['../Model/xgb_advance.pkl']

In [ ]:
temp_df.to_csv('../Datasets/Advance-train.csv')